# Installing all the necessary libraries

In [3]:
%%capture
!conda install -c conda-forge scip -y
!pip install PySCIPOpt
!pip install pyomo
!pip install highspy
!pip install mlflow

# Example Cell To Pass Optimization Parameters

In [4]:
# # # Pipeline parameters
# scenario = 9
# vessel_count = 6
# optimization_type = "throughput"
# max_demurrage_limit = 10

## ML Flow Initiation

In [5]:
import mlflow
mlflow.set_experiment(f"vessel_routing_crude_blending_optimization_scenario_{scenario}_experiment")
mlflow.start_run()

# Load Data


In [6]:
import pandas as pd
import json


def load_all_scenario_data(scenario):

    base_path = f"/lakehouse/default/Files/Scenario {scenario}/"
    with open(f"{base_path}config.json", "r") as f:
     config = json.load(f)

    crude_availability_df = pd.read_csv(base_path + "crude_availability.csv")
    crude_availability = {}
    for _, row in crude_availability_df.iterrows():
        crude_availability \
            .setdefault(row["date_range"], {}) \
            .setdefault(row["location"], {})[row["crude"]] = {
                "volume": int(row["volume"]),
                "parcel_size": int(row["parcel_size"])
            }
    
    
    time_of_travel_df = pd.read_csv(base_path + "time_of_travel.csv")
    
   
    time_of_travel = {
        (row["from"], row["to"]): int(row["time_in_days"])+1
        for _, row in time_of_travel_df.iterrows()
    }
    products_info = pd.read_csv(base_path + "products_info.csv")
    crudes_info_df = pd.read_csv(base_path + "crudes_info.csv")
    crudes = crudes_info_df["crudes"]
    locations = set(time_of_travel_df["from"]) | set(time_of_travel_df["to"])
    source_location = crudes_info_df["origin"].to_list()
    crude_margins = crudes_info_df['margin'].to_list()

    opening_inventory = crudes_info_df['opening_inventory'].to_list()
    opening_inventory_dict = dict(zip(crudes.to_list(), opening_inventory))

    return config, list(crudes), list(locations), time_of_travel, crude_availability, source_location, products_info, crude_margins, opening_inventory_dict



def extract_window_to_days(crude_availability):
    window_to_days = {}

    for window in crude_availability:
        # Split the date range and take only the day parts (ignore month)
        parts = window.split()[0]  # e.g., "1-3"
        if '-' in parts:
            start_day, end_day = map(int, parts.split('-'))
            days = list(range(start_day, end_day + 1))
        else:
            days = [int(parts)]
        window_to_days[window] = days

    return window_to_days


config, crudes, locations, time_of_travel, crude_availability, source_location, products_info, crude_margins, opening_inventory_dict = load_all_scenario_data(scenario)
window_to_days = extract_window_to_days(crude_availability)

# Mapping of the year & month for the optimization

In [7]:
import pandas as pd
month_map = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12, 
}
month_number = month_map[config["schedule_month"]]
year = config["schedule_year"]
start_date = pd.to_datetime(f"{year}-{month_number:02d}-01")

In [8]:
import ast
def extract_products_ratio(df):
    """Convert DataFrame using dictionary comprehension"""
    return {
        (row['product'], crude): ratio
        for _, row in df.iterrows()
        for crude, ratio in zip(ast.literal_eval(row['crudes']), ast.literal_eval(row['ratios']))
    }
products_ratio=extract_products_ratio(products_info)

# Creating pyomo model

In [9]:
from pyomo.environ import *
model = ConcreteModel()

# 2. Creating Sets

In [10]:
INVENTORY_MAX_VOLUME = config["INVENTORY_MAX_VOLUME"] 
MaxTransitions=config["MaxTransitions"] 
model.CRUDES = Set(initialize=crudes)
model.LOCATIONS = Set(initialize=locations)
model.SOURCE_LOCATIONS = Set(initialize=source_location)
config["VESSELS"] = list(range(1,vessel_count+1))
model.VESSELS = Set(initialize=config["VESSELS"])
model.DAYS = RangeSet(config["DAYS"]["start"], config["DAYS"]["end"])
model.BLENDS = Set(initialize=products_info['product'].to_list(),dimen=None)
model.SLOTS = RangeSet(config["DAYS"]["start"],2*config["DAYS"]["end"])

In [11]:
products_capacity = dict(zip(products_info['product'].to_list(), products_info['max_per_day']))
crude_margins_dict = dict(zip(crudes, crude_margins))

model.BCb = Param(model.BLENDS, initialize=products_capacity)
model.BRcb = Param(model.BLENDS, model.CRUDES, initialize=products_ratio, default=0)
model.MRc = Param(model.CRUDES, initialize=crude_margins_dict)

In [12]:
parcel_set = set()
for window, loc_data in crude_availability.items():
    for location, crude_dict in loc_data.items():
        for crude_type, info in crude_dict.items():
            parcel_set.add((location, crude_type, window))
model.PARCELS = Set(initialize=parcel_set, dimen=3)

# 3. Creating Params

In [13]:
parcel_size = {}
for window, loc_data in crude_availability.items():
    for location, crude_dict in loc_data.items():
        for crude_type, info in crude_dict.items():
            key = (location, crude_type, window)
            parcel_size[key] = info["parcel_size"]
            
model.PVp = Param(model.PARCELS, initialize=parcel_size)

In [14]:
def pc_init(model, *p):
    return p[1]
model.PCp = Param(model.PARCELS, initialize=pc_init, within=Any)

In [15]:
model.Travel_Time = Param(model.LOCATIONS, model.LOCATIONS, initialize=time_of_travel)

In [16]:
def pdp_init(model, *p):
    window = p[2]  
    return window_to_days[window]
model.PDp = Param(model.PARCELS, initialize=pdp_init)

In [17]:
def plp_init(model, *p):
    return p[0]
model.PLp = Param(model.PARCELS, within=model.SOURCE_LOCATIONS, initialize=plp_init)

In [18]:
days = list(range(config["DAYS"]["start"], config["DAYS"]["end"] + 1))
capacity_dict = {}

for entry in config['plant_capacity_reduction_window']:
    cap = entry['max_capacity']
    start = entry['start_date']
    end = entry['end_date']
    for day in range(start, end + 1):
        capacity_dict[day] = cap

default_capacity = config['default_capacity']
for day in days:
    capacity_dict.setdefault(day, default_capacity)

model.RCd = Param(model.DAYS, initialize=capacity_dict)

# 4. Decision variables

In [19]:
model.AtLocation = Var(model.VESSELS, model.LOCATIONS, model.DAYS, domain=Binary)
model.Discharge = Var(model.VESSELS, model.DAYS, domain=Binary)
model.Pickup = Var(model.VESSELS, model.PARCELS, model.DAYS, domain=Binary)
model.Inventory = Var(model.CRUDES, model.DAYS, domain=NonNegativeReals)
model.BlendFraction = Var(model.BLENDS, model.SLOTS, domain=NonNegativeReals)
model.DischargeDay = Var(model.VESSELS, domain=PositiveIntegers)
model.Ullage = Var(model.DAYS, domain=NonNegativeReals)

# Auxilary variables

In [20]:
model.LocationVisited = Var(model.VESSELS, model.LOCATIONS, domain=Binary)
model.CrudeInVessel = Var(model.VESSELS, model.CRUDES, domain=Binary)

model.NumGrades12 = Var(model.VESSELS, domain=Binary)
model.NumGrades3 = Var(model.VESSELS, domain=Binary)

model.VolumeDischarged = Var(model.VESSELS, model.CRUDES, model.DAYS, domain=NonNegativeReals)
model.VolumeOnboard = Var(model.VESSELS, model.CRUDES, domain=NonNegativeReals)

model.IsBlendConsumed = Var(model.BLENDS, model.SLOTS, domain=Binary)

model.IsTransition = Var(model.BLENDS, model.SLOTS, domain=Binary)
model.Departure = Var(model.VESSELS, model.LOCATIONS, model.DAYS, domain=Binary)

# 5. Constraints

## 5.1 Vessel travel constraints

In [21]:
# Constraint 1

# Equation - 1
# 1. A vessel can only be at one location on a given day
def vessel_single_location_rule(model, v, d):
    return sum(model.AtLocation[v, l, d] for l in model.LOCATIONS) <= 1
model.VesselSingleLocation = Constraint(model.VESSELS, model.DAYS, rule=vessel_single_location_rule)


# Constraint 2

# Equation - 2
# 2. A vessel cannot visit a location twice. This can be modeled by putting a constraint that for a given vessel and a location, a 1 turning to 0 from day d to d + 1 can only happen once in the whole schedule.
def departure_lower_bound_rule(model, v, l, d):
    if d == model.DAYS[-1]:  # Ensure d+1 is within bounds
        return Constraint.Skip
    else:
        return model.Departure[v, l, d] >= model.AtLocation[v, l, d] - model.AtLocation[v, l, d + 1]
model.DepartureLowerBound = Constraint(model.VESSELS, model.LOCATIONS, model.DAYS, rule=departure_lower_bound_rule)


# Equation - 3
def departure_upper_bound1_rule(model, v, l, d):
    return model.Departure[v, l, d] <= model.AtLocation[v, l, d]
model.DepartureUpperBound1 = Constraint(model.VESSELS, model.LOCATIONS, model.DAYS, rule=departure_upper_bound1_rule)


# Equation - 4
def departure_upper_bound2_rule(model, v, l, d):
    if d == model.DAYS[-1]:
        return Constraint.Skip
    else:
        return model.Departure[v, l, d] <= 1 - model.AtLocation[v, l, d + 1]
model.DepartureUpperBound2 = Constraint(model.VESSELS, model.LOCATIONS, model.DAYS, rule=departure_upper_bound2_rule)


# Equation - 5
def single_departure_per_location_rule(model, v, l):
    return sum(model.Departure[v, l, d] for d in model.DAYS) <= 1
model.SingleDeparturePerLocation = Constraint(model.VESSELS, model.LOCATIONS, rule=single_departure_per_location_rule)


# Constraint 3
#  After departure from a source location the vessel should reach destination location according to the vessel travel time.

# Equation - 6
def enforce_travel_time_rule(model, v, l, d):
    valid_destinations = []
    for l2 in model.LOCATIONS:
        if (l,l2) in time_of_travel:
            travel_time = model.Travel_Time[l, l2]
            arrival_day = d + travel_time
            if arrival_day in model.DAYS:
                valid_destinations.append(model.AtLocation[v, l2, arrival_day])
    if valid_destinations:
        return model.Departure[v, l, d] <= sum(valid_destinations)
    else:
        return Constraint.Skip
model.EnforceTravelTime = Constraint(model.VESSELS, model.SOURCE_LOCATIONS, model.DAYS, rule=enforce_travel_time_rule)

# Constraint 4
# Vessel can’t reach the destination location before the travel time between the locations.

# Equation - 7
def no_early_arrival_rule(model, vessel, source_location, destination_location, start_day, end_day):
    if source_location == destination_location:
        return Constraint.Skip
        
    if (source_location,destination_location) not in time_of_travel:
        return Constraint.Skip

    if end_day-start_day>=model.Travel_Time[source_location, destination_location]:
        return Constraint.Skip
    
    if end_day<=start_day:
        return Constraint.Skip
    
    return model.AtLocation[vessel, source_location, start_day] + model.AtLocation[vessel, destination_location, end_day] <= 1
model.NoEarlyArrival = Constraint(model.VESSELS, model.LOCATIONS, 
                                model.LOCATIONS, model.DAYS, model.DAYS, rule=no_early_arrival_rule)


# Constraint 5
# vessel should not stay at source location after loading activity

# Equation - 8
def depart_after_load_rule(model, v, l, d):
    return model.Departure[v,l,d] <= sum(
        model.Pickup[v, p, d]
        for p in model.PARCELS
        if model.PLp[p] == l
    )
model.DepartAfterLoad = Constraint(model.VESSELS, model.SOURCE_LOCATIONS, model.DAYS, rule=depart_after_load_rule)

## 5.2 Vessel loading constraints

In [22]:
# Constraint 1
# Every vessel has to at least pick one parcel.

# Equation - 9
def atleast_one_parcel_per_vessel(model, vessel):
    return sum(
        model.Pickup[vessel, parcel, day] 
        for parcel in model.PARCELS
        for day in model.DAYS
    ) >= 1
model.AtleastOneParcelPerVessel = Constraint(model.VESSELS, rule=atleast_one_parcel_per_vessel)

# Constraint 2
# A parcel can only be picked up by one vessel.

# Equation - 10
def one_ship_for_one_parcel_pickup(model, *parcel):
    return sum(
        model.Pickup[vessel, parcel, day] 
        for vessel in model.VESSELS
        for day in model.DAYS
    ) <= 1
model.OneVesselParcel = Constraint(model.PARCELS, rule=one_ship_for_one_parcel_pickup)

# Constraint 3
# only one pickup a day for a vessel

# Equation - 11
def one_pickup_per_day(model, v, d):
    return sum(
        model.Pickup[v, parcel, d] 
        for parcel in model.PARCELS
    ) <= 1
model.OnePickupDayVessel = Constraint(model.VESSELS, model.DAYS, rule=one_pickup_per_day)

# Constraint 4
# A pickup can only happen on the day the parcel is available.

# Equation - 12
def pickup_day_limit(model, vessel, *parcel):
    return sum(
        model.Pickup[vessel, parcel, day]
        for day in model.DAYS
        if day not in model.PDp[parcel]
    ) == 0
model.PickupDayLimit = Constraint(model.VESSELS, model.PARCELS, rule=pickup_day_limit)

# Constraint 5
# A vessel can only pickup a parcel if that vessel is in the location where parcel is present.

# Equation - 13
def parcel_location_bound(model, vessel, day, *parcel):
    return model.Pickup[vessel, parcel, day] <= model.AtLocation[vessel, model.PLp[parcel], day]

model.ParcelLocationBound = Constraint(model.VESSELS, model.DAYS, model.PARCELS, rule=parcel_location_bound)


# Constraint 6

# If a vessel is visiting a location, it should at least pick one parcel from there. We introduce an auxiliary variable 
# LocationVisited(v, l) ∈ {0, 1} which is 1 when vessel v visited location l, 0 otherwise.

M = 30
# Equation - 14
def location_visited_constraint_1(model, vessel, location):
    return sum(
        model.AtLocation[vessel, location, day]
        for day in model.DAYS
    ) >= model.LocationVisited[vessel, location]
model.LocationConstraint1 = Constraint(model.VESSELS, model.SOURCE_LOCATIONS, rule=location_visited_constraint_1)

# Equation - 15
def location_visited_constraint_2(model, vessel, location):
    return sum(
        model.AtLocation[vessel, location, day]
        for day in model.DAYS
    ) <= M*model.LocationVisited[vessel, location]
model.LocationConstraint2 = Constraint(model.VESSELS, model.SOURCE_LOCATIONS, rule=location_visited_constraint_2)

# Equation - 16
def location_visited_constraint_3(model, vessel, location):
    return sum(
        sum(
            model.Pickup[vessel, parcel, day]
            for day in model.DAYS
        )
        for parcel in model.PARCELS if model.PLp[parcel] == location
    ) >= model.LocationVisited[vessel, location]
model.LocationConstraint3 = Constraint(model.VESSELS, model.SOURCE_LOCATIONS, rule=location_visited_constraint_3)

# Constraint 7

# A vessel can carry at max 3 different types of crude. For this we will need to introduce 
# an auxiliary variable CrudeInVessel(v, c) ∈ {0, 1} which is 1 if vessel v is carrying crude c, otherwise 0.

M = 30
# Equation - 17
def crude_in_vessel_bound_with_pickup(model, vessel, crude):
    return sum(
        sum(
            model.Pickup[vessel, parcel, day]
            for day in model.DAYS
        )
        for parcel in model.PARCELS if model.PCp[parcel] == crude
    ) >= model.CrudeInVessel[vessel, crude]
model.CrudeInVesselBoundWithPickup = Constraint(model.VESSELS, model.CRUDES, rule=crude_in_vessel_bound_with_pickup)

# Equation - 18
def crude_in_vessel_lower_bound(model, vessel, crude):
    return sum(
        sum(
            model.Pickup[vessel, parcel, day]
            for day in model.DAYS
        )
        for parcel in model.PARCELS if model.PCp[parcel] == crude
    ) <= model.CrudeInVessel[vessel, crude]*M
model.CrudeInVesselLowerBound = Constraint(model.VESSELS, model.CRUDES, rule=crude_in_vessel_lower_bound)

# Equation - 19
def max_3_crudes_limit(model, vessel):
    return sum(
        model.CrudeInVessel[vessel, crude]
        for crude in model.CRUDES
    ) <= 3
model.Max3CrudesLimit = Constraint(model.VESSELS, rule=max_3_crudes_limit)

# Constraint 8

# The max crude volume which a vessel can carry depends on number of types of crude grades on
# that vessel. If the vessel is carrying 1 or 2 grades it can carry 700 Kb and if it is carrying 3 grades
# which is the upper limit, then that is 650 Kb.

# Equation - 20
def crude_group_limit(model, vessel):
    return model.NumGrades12[vessel] + model.NumGrades3[vessel] == 1
model.CrudeGroupLimit = Constraint(model.VESSELS, rule=crude_group_limit)

# Equation - 21
def total_crude_upper_limit(model, vessel, crude):
    return 2*model.NumGrades12[vessel] + 3*model.NumGrades3[vessel] >= sum(
        model.CrudeInVessel[vessel, crude]
        for crude in model.CRUDES
    )
model.TotalCrudeUpperLimit = Constraint(model.VESSELS, model.CRUDES, rule=total_crude_upper_limit)

# Equation - 22
def total_crude_lower_limit(model, vessel, crude):
    return model.NumGrades12[vessel] + 3*model.NumGrades3[vessel] <= sum(
        model.CrudeInVessel[vessel, crude]
        for crude in model.CRUDES
    )
model.TotalCrudeLowerLimit = Constraint(model.VESSELS, model.CRUDES, rule=total_crude_lower_limit)

# Equation - 23
def crude_count_wise_vessel_volume_limit(model, vessel):
    return sum(
        model.PVp[parcel]*sum(
            model.Pickup[vessel, parcel, day]
            for day in model.DAYS
        )
        for parcel in model.PARCELS
    ) <= config['two_parcel_vessel_capacity']*model.NumGrades12[vessel] + config['three_parcel_vessel_capacity']*model.NumGrades3[vessel]
model.CrudeCountWiseVesselVolume = Constraint(model.VESSELS, rule=crude_count_wise_vessel_volume_limit)

## 5.3 Vessel discharge constraints

In [30]:
# Constraint 1
# vessel discharge start day

# Equation - 24
def unique_vessel_discharge_day(model, v):
    return sum(model.Discharge[v, d] for d in model.DAYS) == 1
model.UniqueVesselDischargeDay = Constraint(model.VESSELS, rule=unique_vessel_discharge_day)

# Constraint 2
# discharge should happen in Melaka and should take two days

# Equation - 25
def discharge_at_melaka_rule(model, v, d):
    if d == model.DAYS[-1]: 
        # return model.Discharge[v, d] == 0
        return 2 * model.Discharge[v, d] <= model.AtLocation[v, "Melaka", d]
    else:
        return 2 * model.Discharge[v, d] <= model.AtLocation[v, "Melaka", d] + model.AtLocation[v, "Melaka", d + 1]
model.DischargeAtMelaka = Constraint(model.VESSELS, model.DAYS, rule=discharge_at_melaka_rule)

# Constraint 3 
# no 2 vessels could discharge on the range of given consecutive days 

# Equation - 26
def no_two_vessels_discharge_same_or_adjacent_day_rule(model, d):
    if d == model.DAYS[-1]:  
        # return Constraint.Skip
        return sum(model.Discharge[v, d] for v in model.VESSELS) <= 1
    else:
        return sum(model.Discharge[v, d] + model.Discharge[v, d + 1] for v in model.VESSELS) <= 1
model.NoTwoDischargeSameOrAdjacent = Constraint(model.DAYS, rule=no_two_vessels_discharge_same_or_adjacent_day_rule)

# Constraint 4
# if a vessel discharge completely it cannot be at any other location after discharging

# Equation - 27
def vessel_stops_after_discharge_rule(model, v, l, d1, d2):
    if d2 <= d1 + 1:
        return Constraint.Skip
    return model.AtLocation[v, l, d2] <= 1 - model.Discharge[v, d1]
model.VesselStopsAfterDischarge = Constraint(model.VESSELS, model.LOCATIONS, model.DAYS, model.DAYS, rule=vessel_stops_after_discharge_rule)

# Constraint 5
# creating auxiliary variables to handle the volume unboard & discharged

# Equation - 28
def volume_onboard_rule(model, v, c):
    return model.VolumeOnboard[v, c] == sum(
        model.PVp[p] * sum(model.Pickup[v, p, d] for d in model.DAYS)
        for p in model.PARCELS
        if model.PCp[p] == c
    )
model.VolumeOnboardDef = Constraint(model.VESSELS, model.CRUDES, rule=volume_onboard_rule)



# Equation - 29
def discharge_upper_limit_rule(model, v, c, d):
    return model.VolumeDischarged[v, c, d] <= config['vessel_max_limit'] * model.Discharge[v, d]
model.DischargeUpperLimit = Constraint(model.VESSELS, model.CRUDES, model.DAYS, rule=discharge_upper_limit_rule)


# Equation - 30
def discharge_no_more_than_onboard_rule(model, v, c, d):
    return model.VolumeDischarged[v, c, d] <= model.VolumeOnboard[v, c]
model.DischargeNoMoreThanOnboard = Constraint(model.VESSELS, model.CRUDES, model.DAYS, rule=discharge_no_more_than_onboard_rule)


# Equation - 31
def discharge_lower_bound_rule(model, v, c, d):
    return model.VolumeDischarged[v, c, d] >= model.VolumeOnboard[v, c] - config['vessel_max_limit'] * (1 - model.Discharge[v, d])
model.DischargeLowerBound = Constraint(model.VESSELS, model.CRUDES, model.DAYS, rule=discharge_lower_bound_rule)


## 5.4 Vessel ordering constraints

In [ ]:
# Constarint 1
# If we switch two vessels the results would be exactly the same. This is known as issue
# of symmetry in MILP where the solvers spend more time exploring all symmetric permutations which
# slows down the system. To mitigate this we are adding symmetry breaking constraints.

# Equation 32
# Symmetry constraints
def discharge_day_rule(model, v):
    return model.DischargeDay[v] == sum(d * model.Discharge[v, d] for d in model.DAYS)
model.CalcDischargeDay = Constraint(model.VESSELS, rule=discharge_day_rule)
 
# Equation 33
def symmetry_breaking_rule(model, v):
    if v < len(model.VESSELS):
        return model.DischargeDay[v] + 1 <= model.DischargeDay[v + 1]
    return Constraint.Skip
model.SymmetryBreak = Constraint(model.VESSELS, rule=symmetry_breaking_rule)

##  5.5 Crude blending constraints

In [31]:
# Constraint - 1
# Consume only one blend per slot

# Equation - 34
def is_blend_greater_than_fraction_rule(model, s, *b):
    return model.IsBlendConsumed[b, s] >= model.BlendFraction[b, s]
model.IsBlendVsFraction = Constraint(model.SLOTS, model.BLENDS, rule=is_blend_greater_than_fraction_rule)

# Equation - 35
def one_blend_per_slot_rule(model, s):
    return sum(model.IsBlendConsumed[b, s] for b in model.BLENDS) == 1
model.OneBlendPerSlot = Constraint(model.SLOTS, rule=one_blend_per_slot_rule)



# Constraint - 2
# sum of two fraction of blends should sum up to 1 for a given day on 2 slots

# Equation - 36
def blend_fraction_daily_upper_bound_rule(model, s):
    if s % 2 == 1 and s + 1 in model.SLOTS:
        return sum(model.BlendFraction[b, s] + model.BlendFraction[b, s + 1] for b in model.BLENDS) <= 1
    else:
        return Constraint.Skip
model.BlendFractionDailyBound = Constraint(model.SLOTS, rule=blend_fraction_daily_upper_bound_rule)

# Constraint - 3
# we have to lower down as many transitions as possible 

# Equation - 37
def transition_lower_bound_rule(model, s, *b):
    if s + 1 in model.SLOTS:
        return model.IsTransition[b, s] >= model.IsBlendConsumed[b, s] - model.IsBlendConsumed[b, s + 1]
    else:
        return Constraint.Skip
model.TransitionLowerBound = Constraint(model.SLOTS, model.BLENDS, rule=transition_lower_bound_rule)

# Equation - 38
def transition_upper_bound1_rule(model, s, *b):
    return model.IsTransition[b, s] <= model.IsBlendConsumed[b, s]
model.TransitionUpperBound1 = Constraint(model.SLOTS, model.BLENDS, rule=transition_upper_bound1_rule)

# Equation - 39
def transition_upper_bound2_rule(model, s, *b):
    if s + 1 in model.SLOTS:
        return model.IsTransition[b, s] <= 1 - model.IsBlendConsumed[b, s + 1]
    else:
        return Constraint.Skip
model.TransitionUpperBound2 = Constraint(model.SLOTS, model.BLENDS, rule=transition_upper_bound2_rule)

# Equation - 40
def max_transitions_rule(model):
    return sum(model.IsTransition[b, s] for b in model.BLENDS for s in model.SLOTS) <= MaxTransitions
model.MaxTransitionsConstraint = Constraint(rule=max_transitions_rule)

# Constraint - 4
# refinery capactiy should be in check accoridng to the scnearios

# equation 41
def plant_capacity_rule(model, d):
    return sum(
        model.BCb[b] * (model.BlendFraction[b, 2*d - 1] + model.BlendFraction[b, 2*d])
        for b in model.BLENDS
    ) <= model.RCd[d]

model.PlantCapacityConstraint = Constraint(model.DAYS, rule=plant_capacity_rule)


# Equation 42
def minimum_plant_capacity_production_rule(model, d):
    return sum(
        model.BCb[b] * (model.BlendFraction[b, 2*d - 1] + model.BlendFraction[b, 2*d])
        for b in model.BLENDS
    ) >= config['turn_down_capacity']

model.MinimumPlantCapacityProductionConstraint = Constraint(model.DAYS, rule=minimum_plant_capacity_production_rule)

## 5.6 Inventory constraints

In [33]:

# Constraint 1
# updating the inventory for the each day use and consume 

# Equation 43 to 46 are covered in this 
def inventory_update_rule(model, c, d):
    discharged = 0
    if d <= 5:  
        discharged = 0
    else:
        discharged = sum(model.VolumeDischarged[v, c, d-5] for v in model.VESSELS)

    consumed = sum(
        model.BCb[blend]*model.BRcb[blend,c]*(model.BlendFraction[blend, 2*d-1] + model.BlendFraction[blend, 2*d])
        for blend in model.BLENDS
    )
    if d==1:
        return model.Inventory[c, d] == opening_inventory_dict[c] + discharged - consumed    
    else:
        return model.Inventory[c, d] == model.Inventory[c, d-1] + discharged - consumed
model.InventoryUpdate = Constraint(model.CRUDES, model.DAYS, rule=inventory_update_rule)


def max_inventory_limit(model, day):
    return sum(
        model.Inventory[crude, day]
        for crude in model.CRUDES
    ) <= INVENTORY_MAX_VOLUME
model.MaxInventoryLimit = Constraint(model.DAYS, rule=max_inventory_limit)


# Ullage constraints

# Equation - 47 to 49 are covered in this 
def ullage_update_rule(model, d):
    consumed = sum(
        model.BCb[b] * (model.BlendFraction[b, 2*d - 1] + model.BlendFraction[b, 2*d])
        for b in model.BLENDS
    )
    if d == 1:
        return model.Ullage[d] == INVENTORY_MAX_VOLUME - sum(opening_inventory_dict[c] for c in model.CRUDES) + consumed

    # Only compute discharge if d-1 is in model.DAYS
    discharged = 0
    if (d - 1) in model.DAYS:
        discharged = sum(
            model.VolumeDischarged[v, c, d - 1]
            for v in model.VESSELS
            for c in model.CRUDES
        )

    return model.Ullage[d] == model.Ullage[d - 1] - discharged + consumed
model.UllageUpdate = Constraint(model.DAYS, rule=ullage_update_rule)

## 6. Objective function

### 6.1 Maximizing net profit

In [36]:
# Demurrage from vessels staying at the source locations. If the vessel is not picking up any crude while
# at source location then that day gets counted for demurrage.

# 1.5.1 Maximizing net profit
# Equation - 50
def demurrage_at_source_expr(model):
    return config['demurrage_cost'] * (
            sum(
                model.AtLocation[vessel,location,day] 
                for vessel in model.VESSELS
                for location in model.LOCATIONS
                for day in model.DAYS
                if location != 'Melaka'
        ) - sum(
            model.Pickup[vessel, parcel, day]
            for vessel in model.VESSELS
            for parcel in model.PARCELS
            for day in model.DAYS
        )
    )
model.DeumrrageAtSource = Expression(rule=demurrage_at_source_expr)

# Equation - 51
def demurrage_at_melaka_expr(model):
    return config['demurrage_cost'] * (sum(
        sum(
            model.AtLocation[vessel,'Melaka', day]
            for day in model.DAYS
        ) - 2
        for vessel in model.VESSELS
    ))
model.DemurrageAtMelaka = Expression(rule=demurrage_at_melaka_expr)

# Equation - 52
def total_profit_expr(model):
    return sum(
        model.MRc[crude]*model.BRcb[blend,crude]*model.BCb[blend]*model.BlendFraction[blend,slot]
        for crude in model.CRUDES
        for blend in model.BLENDS
        for slot in model.SLOTS
    )
model.TotalProfit = Expression(rule=total_profit_expr)


### 6.2 Maximizing throughput

In [ ]:
# for Equation 53 we do not cover it as it's just the manual calcualtion of subtracting the Margin - Demurrage as for calculating profit 

# Equation - 54
def total_throughput_expr(model):
    return sum(
        model.BCb[blend]*model.BlendFraction[blend,slot]
        for blend in model.BLENDS
        for slot in model.SLOTS
    )
model.Throughput = Expression(rule=total_throughput_expr)

def net_profit_objective_rule(model):

    return model.TotalProfit - (
        model.DeumrrageAtSource + model.DemurrageAtMelaka
    )

def total_throughput_objective_rule(model):
    return model.Throughput

### Choosing the right optimization function according to optimization_type

In [ ]:
if optimization_type == 'margin':
    model.objective = Objective(rule=net_profit_objective_rule, sense=maximize)
elif optimization_type == 'throughput':
    # apply demurrage day limit constraint
    model.DemurrageLimitConstraint = Constraint(expr=model.DeumrrageAtSource + model.DemurrageAtMelaka<=max_demurrage_limit*config["demurrage_cost"])
    model.objective = Objective(rule=total_throughput_objective_rule, sense=maximize)
else:
    raise NotImplemented

# Solver configs

In [38]:
solver_config = {
    "name": "highs",
    "options": {
        "threads": 4,
        "presolve": "on",
        "mip_rel_gap": 0.01
    }
}

In [39]:
def get_enabled_solver(solver_cfg):
    """Get the first enabled solver from config and apply all options directly."""
    
    
    solver_name = solver_cfg.get("name")

    solver = SolverFactory(solver_name)

    print(f"Using solver: {solver_name}")
 
    solver.options["time_limit"] = config["solver_time_limit_seconds"]
    for key, value in solver_cfg.get("options", {}).items():
        solver.options[key] = value

    print(f"Solver options: {solver.options}")
    return solver

# Optimization Process

In [40]:
solver = get_enabled_solver(solver_config)

In [41]:
import sys
from contextlib import redirect_stdout
from pyomo.contrib.solver.common.util import NoFeasibleSolutionError

dir_path = f"/lakehouse/default/Files/ExperimentResult/Scenario {scenario}/"
os.makedirs(dir_path, exist_ok=True)
if optimization_type == 'throughput':
    model_log_file_path = f'/lakehouse/default/Files/ExperimentResult/Scenario {scenario}/{optimization_type}_Optimization_log_{vessel_count}_vessels_{config["DAYS"]["end"]}_days_{MaxTransitions}_transitions_{max_demurrage_limit}_demurrages.txt'
else:
    model_log_file_path = f'/lakehouse/default/Files/ExperimentResult/Scenario {scenario}/{optimization_type}_Optimization_log_{vessel_count}_vessels_{config["DAYS"]["end"]}_days_{MaxTransitions}_transitions.txt'


try:
    with open(model_log_file_path, "w") as f:
        with redirect_stdout(f):
            results = solver.solve(model, tee=True)
except NoFeasibleSolutionError as e:
    print("Error Ocurred: There is no feasible solution for given dataset and configuration.")
except Exception as e:
    print("Error Ocurred:", str(e))

In [43]:
days=[]
Final_Product=[]
Quantity_produced=[]
profit_each_slot=[]
slots=[]
inventory=[]
ullage = []
crude_blended = {c: [] for c in model.CRUDES}
crude_available = {c: [] for c in model.CRUDES}


for slot in model.SLOTS:
    slots.append(slot)

    if (slot+1)%2==0:
        day=int((slot+1)/2)
    days.append(day)
    total_profit = 0

    for blend in model.BLENDS:

        if value(model.IsBlendConsumed[blend, slot]) > 0.5:

            Final_Product.append(blend)
            produced = value(model.BlendFraction[blend,slot])*value(model.BCb[blend])
            Quantity_produced.append(produced)
            inventory_total = 0
           
            for crude in model.CRUDES:

                blended_amount = value(model.BCb[blend]) * value(model.BRcb[blend,crude]) * value(model.BlendFraction[blend , slot])
                profit=model.MRc[crude]*blended_amount
                crude_blended[crude].append(blended_amount)
                inv = value(model.Inventory[crude, day])
                crude_available[crude].append(inv)
                inventory_total += inv
                total_profit += profit
                
            inventory.append(inventory_total)
    ullage.append(value(model.Ullage[day]))
    profit_each_slot.append(total_profit)   
    

records=[]
for i in range(len(slots)):
    record = {
        "Date": pd.to_datetime(start_date) + pd.Timedelta(days=days[i]-1), 
        "Slot": slots[i],
        "Final Product": Final_Product[i],
        "Quantity Produced": round(Quantity_produced[i]/1000, 1),
        **{f"Crude {c} Available": round(crude_available[c][i]/1000, 1) for c in model.CRUDES},
        **{f"Crude {c} Blended": round(crude_blended[c][i]/1000, 1) for c in model.CRUDES},
        "Inventory Available": round(inventory[i]/1000, 1),
        "Ullage": round(ullage[i]/1000, 1),
        "Profit":profit_each_slot[i],
        "Flag": "Optimization"
    }
    records.append(record)
df = pd.DataFrame(records)

slot=[]
for i in df['Slot']:
    if i %2==0:
        slot.append(2)
    else:
        slot.append(1)
df['Slot']=slot
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
def reduce_rows(group):
    print(group)
    if (group["Quantity Produced"] == 0).sum() == 1:
        # Keep the non-zero row, force slot = 1
        print("yes", group['Date'].to_list())
        row = group[group["Quantity Produced"] != 0].copy()
        row.loc[:, "Slot"] = 1
        return row
    else:
        return group

combined_df_reduced = df.groupby(["Date","Flag"], group_keys=False).apply(reduce_rows).reset_index(drop=True) 

In [46]:
records = []

for v in model.VESSELS:
    is_vessel_started = False
    is_vessel_terminated = False
    is_at_melaka = 0
    last_port_location = None
    pending_sailing_records = []
    crude_loaded = {}

    for d in model.DAYS:
        at_location = False
        activity_name_list = []
        location_visited = None
        is_loading = 0
        is_unloading = 0

        for l in model.LOCATIONS:
            if value(model.AtLocation[v, l, d]) > 0.5:
                at_location = True
                location_visited = l

                # Update last_port_location when vessel is at port
                last_port_location = l

                if not is_vessel_started:
                    activity_name_list.append("Arrival T")
                    is_vessel_started = True

                for p in model.PARCELS:
                    if value(model.Pickup[v, p, d]) > 0.5:
                        crude_type = p[1]
                        crude_volume_carried = parcel_size[p]
                        crude_loaded[f"{crude_type} Volume"] = crude_volume_carried
                        activity_name_list.append("Loading")
                        is_loading = 1
                        break
                

                if l == "Melaka" and is_at_melaka == 0:
                    activity_name_list.append("Arrival M")
                    is_at_melaka = 1

                if value(model.Discharge[v, d]) > 0.5:
                    activity_name_list.append("Discharge")
                    is_unloading = 1
            
                if (d > 1) and value(model.Discharge[v, d-1]) > 0.5:
                    activity_name_list.append("Discharge")
                    is_vessel_terminated = True
                    is_unloading = 1

                if 'Loading' not in activity_name_list and "Discharge" not in activity_name_list:
                    activity_name_list.append("Demurrage")

        if is_vessel_started and not is_vessel_terminated and not at_location:
            activity_name_list.append("Sailing")

        # Find next port when sailing
        next_port_location = None
        if not at_location:
            # Look ahead to find first future location
            for future_d in range(d + 1, max(model.DAYS) + 1):
                for l_future in model.LOCATIONS:
                    if value(model.AtLocation[v, l_future, future_d]) > 0.5:
                        next_port_location = l_future
                        break
                if next_port_location:
                    break

        # Decide Last Port display
        if at_location:
            last_port_display = location_visited
            # Update any pending sailing records now that we know next port
            for rec in pending_sailing_records:
                rec["Last Port"] = f"{rec['Last Port'].split('--')[0]}--{location_visited}"
                records.append(rec)
            pending_sailing_records.clear()
        elif not at_location and last_port_location and next_port_location:
            last_port_display = f"{last_port_location}--{next_port_location}"
        else:
            last_port_display = "Unknown"

        for activity_name in activity_name_list:
            if activity_name == "Demurrage":
                demurrage_activity = 1
            else:
                demurrage_activity = 0
            record = {
                "Activity Date": pd.to_datetime(start_date) + pd.Timedelta(days=d - 1),
                "Activity Name": activity_name,
                "Activity End Date": pd.to_datetime("2024-10-01") + pd.Timedelta(days=d),
                "Vessel ID": v,
                "Last Port": last_port_display,
                **crude_loaded,
                "is_at_Melaka": is_at_melaka,
                "is Demurrage Day": demurrage_activity,
                "is_crude_unloading_day": is_unloading,
                "is_loading": is_loading,
                "Scenario Id": f"Scenario {scenario}"
            }

            if activity_name == "Sailing":
                # Store temporarily to update once we know next port
                pending_sailing_records.append(record)
            else:
                records.append(record)
vessel_df = pd.DataFrame(records)

In [47]:
base_path = f"/lakehouse/default/Files/ExperimentResult/Scenario {scenario}/"
os.makedirs(base_path, exist_ok=True)
if optimization_type == 'throughput':
    crude_blending_filename = f'crude_blending_{optimization_type}_optimization_{vessel_count}_vessels_{config["DAYS"]["end"]}_days_{MaxTransitions}_transitions_{max_demurrage_limit}_demurrages.csv'
    vessel_routing_filename = f'vessel_routing_{optimization_type}_optimization_{vessel_count}_vessels_{config["DAYS"]["end"]}_days_{MaxTransitions}_transitions_{max_demurrage_limit}_demurrages.csv'
else:
    crude_blending_filename = f'crude_blending_{optimization_type}_optimization_{vessel_count}_vessels_{config["DAYS"]["end"]}_days_{MaxTransitions}_transitions.csv'
    vessel_routing_filename = f'vessel_routing_{optimization_type}_optimization_{vessel_count}_vessels_{config["DAYS"]["end"]}_days_{MaxTransitions}_transitions.csv'
combined_df_reduced.to_csv(base_path+crude_blending_filename)
vessel_df.to_csv(base_path+vessel_routing_filename)

# Dumping Model To Pickle File

In [48]:
import pickle
if optimization_type == 'throughput':
    model_file_name = f'/lakehouse/default/Files/ExperimentResult/Scenario {scenario}/{optimization_type}_optimization_{vessel_count}_vessels_{config["DAYS"]["end"]}_days_{MaxTransitions}_transitions_{max_demurrage_limit}_demurrages.pkl'
else:
    model_file_name = f'/lakehouse/default/Files/ExperimentResult/Scenario {scenario}/{optimization_type}_optimization_{vessel_count}_vessels_{config["DAYS"]["end"]}_days_{MaxTransitions}_transitions.pkl'
with open(model_file_name,'wb') as fp:
    pickle.dump(model, fp)

# Data Outputs After Optimization

In [ ]:
from pyomo.environ import value

total_throughput = combined_df_reduced['Quantity Produced'].sum()
total_margin = combined_df_reduced['Profit'].sum()
average_throughput = total_throughput/config["DAYS"]["end"]
average_margin = total_margin/config["DAYS"]["end"]

print("total_throughput:", total_throughput)
print("total_margin:", total_margin)
print("average_throughput:", average_throughput)
print("average_margin:", average_margin)
print(f"Total Demurrage at melaka: {value(model.DemurrageAtMelaka)}")
print(f"Total Demurrage at source: {value(model.DeumrrageAtSource)}")

# Logging important parameters through MLFOW

In [50]:

mlflow.log_param("crude_blending_output_path", base_path+crude_blending_filename)
mlflow.log_param("vessel_routing_output_path", base_path+vessel_routing_filename)

mlflow.log_param("vessel_count", vessel_count)
mlflow.log_param("optimization_type", optimization_type)
mlflow.log_param("total_throughput", total_throughput)
mlflow.log_param("total_margin", total_margin)
mlflow.log_param("average_throughput", average_throughput)
mlflow.log_param("average_margin", average_margin)

In [51]:
mlflow.end_run()